# RMSE

## Reading

[Chapter 15: 15.4 - 15.6](https://inferentialthinking.com/chapters/15/Prediction.html)

In the previous notebook we developed formulas for the slope and intercept of the regression line for data that cluster uniformly around the regression line. In this notebook we'll see that what we learned also works with data that cluster in other ways around the regression line. More importantly, we'll also learn how to evaluate the regression line.

---

## Root Mean Squared Error

First we look at the dataset from the textbook, which works with data from the _Little Women_ novel that we used in Module 1.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url = 'https://raw.githubusercontent.com/DeAnzaDataScience/CIS11/refs/heads/main/datasets_notes/little_women.csv'
little_women = pd.read_csv(url)
print("Number of rows or chapters:", len(little_women))
print("First 5 rows:")
little_women.head()


Each row of the dataset is for one chapter in the book. The row contains the number of characters (letters, numbers, punctuation, spaces) and the number of periods in the chapter.

We want to see if we can predict the number of characters from the number of periods. This means the `x` data is the number of periods, and the  $\hat{y}$ data is the number of predicted characters.

We plot the number of periods vs the number of characters.

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(little_women['Periods'], little_women['Characters'])
plt.xlabel('Periods')
plt.ylabel('Characters')
plt.grid()
plt.show()

We see that the data values are not uniformly clustered around a regression line, but it does look like a linear, positive correlation.

We find the correlation coefficient `r` by bringing in the `standard_units` and `correlation` functions from the previous notebook.

In [ ]:
def standard_units(array):
    return (array - np.mean(array))/np.std(array)

def correlation(array1, array2):
    return np.mean(standard_units(array1) * standard_units(array2))

r = correlation(little_women['Periods'], little_women['Characters'])
print("r:", r)

There is a high correlation of 0.92.

Next we bring in the `slope` and `intercept` functions from the previous notebook to find the slope and intercept that define the regression line.

In [ ]:
def slope(x, y):
    return correlation(x, y) * np.std(y) / np.std(x)

def intercept(x, y):
    return np.mean(y) - slope(x, y) * np.mean(x)

book_slope = slope(little_women['Periods'], little_women['Characters'])
print("slope:", book_slope)
book_intercept = intercept(little_women['Periods'], little_women['Characters'])
print("intercept:", book_intercept)

Recalling what the slope of the regression line means:
> The slope shows that on average, there are about 87 characters for every period.


We plot the prediction line in the scatterplot.

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(little_women['Periods'], little_women['Characters'])
plt.plot(little_women['Periods'], book_slope * little_women['Periods'] + book_intercept, color='green')
plt.xlabel('Periods')
plt.ylabel('Characters')
plt.grid()
plt.show()

As we noted in the previous notebook, the regression line is the best fit line, it doesn't go through all the data points.

In the plot above, the blue points are the actual `y` values or the actual number of characters, and the green regression line is the $\hat{y}$ values or the predicted number of characters.

We now calculate the predicted number of characters (the $\hat{y}$ value) from the number of periods (the `x` value).

In [ ]:
Predicted_Characters = np.round(book_slope * little_women['Periods'] + book_intercept, 1)

Then we find the difference or error between the predicted_characters and the actual number of characters.

In [ ]:
Error = little_women['Characters'] - Predicted_Characters

Last we add these 2 results into the original DataFrame.

In [ ]:
little_women['Predicted_Characters'] = Predicted_Characters
little_women['Error'] = Error
print("First 5 rows:")
little_women.head()

Now we plot the error for some of the data points on the scatterplot to illustrate them.

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(little_women['Periods'], little_women['Characters'])
plt.plot(little_women['Periods'], book_slope * little_women['Periods'] + book_intercept, color='green')
plt.xlabel('Periods')
plt.ylabel('Characters')
plt.grid()

# You don't need to write the code below.
# This code shows the error on the plot for discussion purpose.

# Select 5 data points
selected_points = little_women.iloc[[22, 0, 40, 24, 46]]
# Add vertical lines for the error
for index, row in selected_points.iterrows():
    plt.plot([row['Periods'], row['Periods']], [row['Characters'], row['Predicted_Characters']], color='red', linestyle='-', linewidth=1.5)

plt.show()

We see that out of 5 data points that we chose randomly, 4 are a distance away from the linear progression line, as shown in red. The red lines are the `Error` values asssociated with these data points. The 5th data point is on the regression line or is so close to it such that there's no red error line shown.

Since most data points are some distance away from the regression line, we want to find their average distance from the line.

To do this, we don't want to do a simple averaging, which is to add the `Error` values and divide the sum by the total number of data values. This is because some of the `Error` values are positive and others are negative. If we add the positives and negatives together, they effectively "cancel" each other out, giving us a false average error that might be too low. For example, if we have -5 and +5 as the `Error` values for 2 data points, if we add them together we'll end up with 0 average error.

Instead we'll use the same method as the calculation of the standard deviation:
- square the error values to remove the negative sign
- find the average of all the squares
- take the square root to 'undo' the squaring of the first step

The resulting average error is called the root mean squared error or RMSE. The long name describes how the errors are processsed: squared, then take their mean, and take the square root.

In [ ]:
rmse = np.sqrt(np.mean(little_women['Error']**2))
print("RMSE:", rmse)
print("Average number of characters:", np.mean(little_women['Characters']))
print("Relative RMSE percent:", round(rmse / np.mean(little_women['Characters']) * 100))

The error in prediction is about $\pm 2700$ characters out of an average of about 21,700 characters per chapter. This is about 12% error.

In general, a rough guideline for relative RMSE error is:
- < 10%: excellent
- 10 - 20%: good
- 20 - 30%: fair
- \> 30%: poor

Mathematically, among all straight lines that can be drawn or calculated to estimate the $\hat{y}$ values, the regression line is the unique line that minimizes the root mean squared error. This is why the regression line is also called the _least squares_ line.

---

## The Residuals

Since the linear regression line is the best-fit line and not an exact-fit line, usually there is a distance between the actual `y` value and the predicted $\hat{y}$ value, which is considered the error in the prediction.

The error in the prediction is also called the _residual_. In the _Little Women_ scatterplot above, the residuals are the red lines shown in the plot. While we can plot the residual for each data point with a red line, it's easy to imagine that all the red lines from all the data points would overlap each other, making it difficult to make good observations of them.

Instead, we can draw residual plots to have a better visualization of the errors.

We start with the same dataset of the parent and child heights.

In [ ]:
url = "https://raw.githubusercontent.com/DeAnzaDataScience/CIS11/refs/heads/main/datasets_notes/galton.csv"
family = pd.read_csv(url)
display(family.head())

Just as in the previous dataset, we create a new DataFrame with the `midparentHeight` and `childHeight` column.

In [ ]:
heights = family.loc[:, ['midparentHeight', 'childHeight']].copy()
display(heights.head())

Then we bring in the `slope` and `intercept` function from the previous notebook and use them to build a new `fit` function that produces the predicted child height.

In [ ]:
def slope(x, y):
    return correlation(x, y) * np.std(y) / np.std(x)

def intercept(x, y):
    return np.mean(y) - slope(x, y) * np.mean(x)

def fit(x, y):
    return slope(x, y) * x + intercept(x, y)

Using the `fit` function, we add 2 new columns for the `heights` DataFrame: predictHeight and Error

In [ ]:
heights['predictHeight'] = fit(heights['midparentHeight'], heights['childHeight'])
heights['Error'] = heights['childHeight'] - heights['predictHeight']
display(heights.head())

We now plot the `midParentHeight` vs the actual `childHeight` in a scatterplot, and then add the linear regression line from the `predictHeight`.

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(heights['midparentHeight'], heights['childHeight'], alpha=0.6)
plt.plot(heights['midparentHeight'], heights['predictHeight'], color='green', linewidth=2)
plt.xlabel('Midparent Height')
plt.ylabel('Child Height')
plt.grid()
plt.show()

It's easy to imagine that if we plotted a red error line from each blue data point to the regression line, it would be difficult to see the errors or residuals.

Instead, we use a residual plot, which is a plot of the `midparentHeight` (`x` values) vs the `Error`($\hat{y}$ values).

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(heights['midparentHeight'], heights['Error'], alpha=0.6, color='firebrick')
plt.plot(heights['midparentHeight'], np.zeros_like(heights['midparentHeight']),
         color='purple', linewidth=2)
plt.xlabel('Midparent Height')
plt.ylabel('Error or Residual')
plt.grid()
plt.show()

We make the following observations, which are typical of a good linear regression:
-  The residuals are centered around 0 on the y-axis. This makes sense since 0 error is the ideal.
- The spread is about the same above and below the horizontal line at y=0, for the majority of x values.


---

### Use Case for Residual Plots

**Detecting Non Linear Correlation**

Residual plots can be used to detect when a correlation is not linear.

We use the same dataset as the textbook, which works with the age and length of a sea mammal called a dugong.

In [ ]:
url = "https://raw.githubusercontent.com/DeAnzaDataScience/CIS11/refs/heads/main/datasets_notes/dugong.csv"
dugong = pd.read_csv(url)
display(dugong.head())

As the dugong's age from birth gets higher, we expect its size or length measurement to be higher also. If we can measure the length of a dugong, can we predict its age?

We display the plot for the `Length` vs `Age`.

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(dugong['Length'], dugong['Age'], alpha=0.6)
plt.xlabel('Length')
plt.ylabel('Age')
plt.grid()
plt.show()

It looks like there is mostly positive correlation, but it might be difficult to fit a line along the data points.

We calculate the regression line and plot it.

In [ ]:
predicted_length = fit(dugong['Length'], dugong['Age'])
plt.figure(figsize=(4,4))
plt.scatter(dugong['Length'], dugong['Age'], alpha=0.6)
plt.plot(dugong['Length'], predicted_length, color='green', linewidth=2)
plt.xlabel('Length')
plt.ylabel('Age')
plt.grid()
plt.show()

It looks like there are errors at the larger lengths, and in the middle range of length, the majority of the data points are below the regression line.

We look at the residual plot.

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(dugong['Length'], dugong['Age'] - predicted_length, alpha=0.6, color='firebrick')
plt.plot(dugong['Length'], np.zeros_like(dugong['Length']), color='purple', linewidth=2)
plt.xlabel('Length')
plt.ylabel('Error or Residual')
plt.grid()
plt.show()

We can see that across the `Length`, the spread is not the same above and below the y=0 line.
- The residuals are above the y=0 line for the lowest lengths.
- They mostly drop below the line in the middle range.
- They are above the line again at the highest lengths.

This means the regression is not a line but more of a curve.

**Detecting Uneven Spread**

Residual plots can be used to detect uneven spread of errors, such as when the errors are higher for a specific range of data in the dataset. The term for uneven spread is _heteroscedasticity_, so we can say that residual plots can be used to detect heteroscedasticity.

We use the dataset of hybrid cars in the US that we used in the previous notebook.

In [ ]:
url = "https://raw.githubusercontent.com/DeAnzaDataScience/CIS11/refs/heads/main/datasets_notes/hybrid.csv"
hybrid = pd.read_csv(url)
display(hybrid.head())

We plot the `mpg` vs `acceleration` and recall that it's not a linear correlation: the `mpg` drops as `acceleration` capability increases, but the `mpg` drop flattens out at the highest `acceleration`.

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(hybrid["acceleration"], hybrid["mpg"], alpha=0.6)
plt.xlabel("acceleration")
plt.ylabel("mpg")
plt.grid()
plt.show()

We calculate the linear regression line and add it to the plot.

In [ ]:
linear_regression_line = fit(hybrid["acceleration"], hybrid["mpg"])
plt.figure(figsize=(4,4))
plt.scatter(hybrid["acceleration"], hybrid["mpg"], alpha=0.6)
plt.plot(hybrid["acceleration"], linear_regression_line, color='green', linewidth=2)
plt.xlabel("acceleration")
plt.ylabel("mpg")
plt.grid()
plt.show()

Observing the plot above, we can tell that the linear regression line doesn't fit the data points very well at the low end of the acceleration capability.

The residual plot agrees with our observation.

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(hybrid["acceleration"], hybrid["mpg"] - linear_regression_line,
            alpha=0.6, color='firebrick')
plt.plot(hybrid["acceleration"], np.zeros_like(hybrid["acceleration"]),
         color='purple', linewidth=2)
plt.xlabel("acceleration")
plt.ylabel("Error or Residual")
plt.grid()
plt.plot()

Sure enough, the residual plot shows that the residual spread is not the same above and below 0 for all acceleration capabilities. There is more error or more spread at the lower end.

---

## Properties of the Residuals

1. <u>The residuals and the predicted variables are not correlated.</u><br>
This means that the differences between actual data and predicted data (the residuals) are random and contain no pattern related to the predicted values.

We check the above property by finding the correlation between the residuals and predicted values of the 3 examples above.

First we find the correlation of the predicted child's height `predictHeight` and its residuals.

In [ ]:
print("r = ", round(correlation(heights['predictHeight'], heights['Error']), 2))

Likewise, we find the correlation of the dugong's `predicted_length` and its residuals.



In [ ]:
print("r = ", round(correlation(predicted_length, dugong['Age'] - predicted_length), 2))

And we find the correlation of the car's `mpg` and its residuals.

In [ ]:
print("r =", round(correlation(linear_regression_line, hybrid["mpg"] - linear_regression_line), 2))

2. <u>The average of the residuals is 0.</u><br>
Since the linear regression line is the best-fit line, the positive differences between actual and predicted values and the negative differences between actual and predicted values will 'cancel' each other out. Therefore their sums will be 0.

We check the above property with the 3 examples above

In [ ]:
print("Average of the residuals for heights:", round(np.mean(heights['Error']), 2))
print("Average of the residuals for dugong:", round(np.mean(dugong['Age'] - predicted_length), 2))
print("Average of the residuals for cars:", round(np.mean(hybrid["mpg"] - linear_regression_line), 2))

3. <u>The ratio of SD of residuals to SD of `y` is: <br>
$$
\frac{SD_{residual}}{SD_{y}} =
\sqrt{1 - r^2}
$$</u><br>
where SD is the standard deviation and `r` is the correlation coefficient.<br>

Looking at the "perfect" values for `r`, which is when `r` is 1 or -1, we can see that the ratio is true. Recall from the previous notebook that when `r` is 1 or -1, the data points line up in a straight line in the scatterplot.
- On the left side of the equation: If the data line up in a straight line, it means that the regression line would fit perfectly with the data points and the SD of the residuals would be 0.<br>
- On the right side of the equation: Evaluating the square root above with `r` being 1 or -1, the result is also 0.

For actual `r` values that are between 0 and 1, we check the observed SD ratio with the 3 datasets above.

In [ ]:
# predict child height from parent height

SD_y = np.std(heights['childHeight'])
SD_residual = np.std(heights['Error'])
r = correlation(heights['midparentHeight'], heights['childHeight'])
print("Heights")
print("SD ratio:", round(SD_residual/SD_y, 4))
print("square root:", round(np.sqrt(1 - (r**2)),4))

In [ ]:
# predict dugong age from length

SD_y = np.std(dugong['Age'])
SD_residual = np.std(dugong['Age'] - predicted_length)
r = correlation(dugong['Length'], dugong['Age'])
print("Dugong")
print("SD ratio:", round(SD_residual/SD_y, 4))
print("square root:", round(np.sqrt(1 - (r**2)),4))

In [ ]:
# predict hybrid car mpg from acceleration

SD_y = np.std(hybrid["mpg"])
SD_residual = np.std(hybrid["mpg"] - linear_regression_line)
r = correlation(hybrid['acceleration'], hybrid["mpg"])
print("Cars")
print("ratio:", round(SD_residual/SD_y, 4))
print("square root:", round(np.sqrt(1 - (r**2)),4))

---

In this notebook we learn that when 2 features of a dataset have linear correlation, then a linear regression line can be calculated to best fit the correlated data. This regression line is the line of predicted data.

The regression line is optimized so that the residuals, or the differences between predicted and actual data, are minimized and the prediction can be as accurate as possible.

The ability to predict a characteristic of the dataset with reasonable accuracy is the data science foundation on which machine learning models are built. Moving beyond calculating the linear regression of a dataset, data scientists use the same simulation and estimation techniques that we've learned to build _linear regression models_ that work with any dataset.